In [30]:
import pandas as pd

In [31]:
debris = pd.read_csv("space_decay.csv")

In [32]:
for col in debris.columns:
    if len(debris[col].unique()) == 1:
        print(f"{col}: {debris[col].unique()}")

CCSDS_OMM_VERS: [2]
COMMENT: ['GENERATED VIA SPACE-TRACK.ORG API']
ORIGINATOR: ['18 SPCS']
CENTER_NAME: ['EARTH']
REF_FRAME: ['TEME']
TIME_SYSTEM: ['UTC']
MEAN_ELEMENT_THEORY: ['SGP4']
EPHEMERIS_TYPE: [0]
CLASSIFICATION_TYPE: ['U']
ELEMENT_SET_NO: [999]
DECAY_DATE: [nan]


In [33]:
debris = debris.drop(columns=["CCSDS_OMM_VERS", "COMMENT", "CREATION_DATE", "ORIGINATOR","OBJECT_NAME","OBJECT_ID", "CENTER_NAME", "REF_FRAME", "TIME_SYSTEM", "MEAN_ELEMENT_THEORY", "CLASSIFICATION_TYPE", "EPHEMERIS_TYPE", "ELEMENT_SET_NO", "DECAY_DATE", "EPOCH", "NORAD_CAT_ID", "ELEMENT_SET_NO", "COUNTRY_CODE", "LAUNCH_DATE", "FILE", "TLE_LINE0", "TLE_LINE1", "TLE_LINE2"])

In [34]:
debris = debris.drop("GP_ID", axis=1)

In [35]:
debris

,MEAN_MOTION,ECCENTRICITY,INCLINATION,RA_OF_ASC_NODE,ARG_OF_PERICENTER,MEAN_ANOMALY,REV_AT_EPOCH,BSTAR,MEAN_MOTION_DOT,MEAN_MOTION_DDOT,SEMIMAJOR_AXIS,PERIOD,APOAPSIS,PERIAPSIS,OBJECT_TYPE,RCS_SIZE,SITE
0,2.921700,0.652893,7.7156,90.2410,243.1216,38.7796,23588,0.024341,8.830000e-06,0.0,20668.618,492.864,27784.871,796.095,DEBRIS,MEDIUM,FRGUI
1,13.754973,0.003072,82.9193,299.1120,158.9093,201.3337,50124,0.000066,7.900000e-07,0.0,7358.038,104.689,1002.507,957.299,DEBRIS,SMALL,PKMTR
2,1.038224,0.023739,12.1717,16.5368,250.1248,146.2900,7798,0.000000,-1.650000e-06,0.0,41197.852,1386.984,35797.696,33841.738,PAYLOAD,LARGE,SRI
3,14.775907,0.006062,98.4781,8.7205,37.3771,323.1632,10204,0.000316,2.265000e-05,0.0,7015.076,97.456,679.465,594.417,DEBRIS,SMALL,TSC
4,14.724482,0.006226,98.4232,122.0724,345.1605,27.6061,9622,0.000408,2.610000e-05,0.0,7031.400,97.796,697.039,609.491,DEBRIS,SMALL,TSC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14367,15.465781,0.001190,99.0098,214.6827,219.1613,140.8771,37922,0.006809,3.548080e-03,0.0,6804.882,93.109,434.844,418.650,TBA,NaN,NaN
14368,14.967557,0.005560,99.0272,141.1159,76.8926,283.8489,32962,0.001777,2.095000e-04,0.0,6955.066,96.208,615.599,538.263,TBA,NaN,NaN
14369,12.909691,0.056673,31.9916,262.9339,162.7824,199.2858,9113,-0.000175,-6.800000e-07,0.0,7675.817,111.544,1732.692,862.672,TBA,NaN,NaN
14370,14.245319,0.005822,98.6758,348.3068,187.0288,173.0081,9893,0.000311,6.900000e-06,0.0,7188.204,101.086,851.919,768.219,TBA,NaN,NaN


In [36]:
debris.describe()

,MEAN_MOTION,ECCENTRICITY,INCLINATION,RA_OF_ASC_NODE,ARG_OF_PERICENTER,MEAN_ANOMALY,REV_AT_EPOCH,BSTAR,MEAN_MOTION_DOT,MEAN_MOTION_DDOT,SEMIMAJOR_AXIS,PERIOD,APOAPSIS,PERIAPSIS
count,14372.000000,14372.000000,14372.000000,14372.000000,14372.000000,14372.000000,14372.000000,14372.000000,14372.000000,1.437200e+04,14372.000000,14372.000000,14372.000000,14372.000000
mean,12.463847,0.067168,74.354208,182.353111,165.977199,191.015595,32960.199068,-0.000572,0.000049,2.307751e-07,10636.185210,223.521557,5721.060178,2795.040245
std,4.509305,0.181547,29.626780,116.717713,104.461866,109.873807,28449.542765,0.032239,0.002391,1.485939e-05,9580.627227,446.517041,13016.299008,7943.045264
min,0.051783,0.000005,0.001400,0.020000,0.008200,0.004800,0.000000,-0.796460,-0.014873,-4.210300e-05,6542.850000,87.783000,183.393000,73.279000
25%,13.452144,0.000725,53.055300,71.987350,77.363475,90.001600,5832.250000,0.000058,0.000001,0.000000e+00,6956.246750,96.232750,599.876000,546.115000
50%,14.277052,0.003416,86.369200,189.460250,150.483900,206.957400,28944.500000,0.000272,0.000010,0.000000e+00,7177.549000,100.861000,843.686000,702.433500
75%,14.963745,0.013505,98.565125,292.484750,258.328550,284.163200,61422.250000,0.000834,0.000032,0.000000e+00,7468.055000,107.046000,1207.919500,831.959750
max,16.404097,0.897218,144.586200,359.967900,359.989900,359.981900,99945.000000,0.595600,0.225321,1.654800e-03,304049.635000,27808.429000,372136.402000,224547.015000
